In [1]:
import pickle
import sktime
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sktime.classification.deep_learning.cnn import CNNClassifier
from sktime.classification.deep_learning.cntc import CNTCClassifier
from sktime.classification.deep_learning.fcn import FCNClassifier
from sktime.classification.deep_learning.inceptiontime import InceptionTimeClassifier
from sktime.classification.deep_learning.lstmfcn import LSTMFCNClassifier
from sktime.classification.deep_learning.macnn import MACNNClassifier
from sktime.classification.deep_learning.mcdcnn import MCDCNNClassifier
from sktime.classification.deep_learning.mlp import MLPClassifier
from sktime.classification.deep_learning.resnet import ResNetClassifier
from sktime.classification.deep_learning.rnn import SimpleRNNClassifier
from sktime.classification.deep_learning.tapnet import TapNetClassifier
import numpy as np

In [2]:
file = open("",'rb')
p = pickle.load(file)

In [3]:
X_train = p['tx'][:,:,:3]
y_train = p['ty']
X_valid = p['vx'][:,:,:3]
y_valid = p['vy']
y_columns_walk_other = p['y_columns_walk_other']
X_columns_walk_other = p['x_columns_walk_other'][:,:,:3]
X_train = X_train.transpose(0, 2, 1)
X_valid = X_valid.transpose(0, 2, 1)

X_valid_columns_walk_other = X_columns_walk_other

pre_name = "model_20240404_"
def convertH5TFlite(name):
    model = tf.keras.models.load_model(name, compile=False)
    
    # Converting a tf.Keras model to a TensorFlow Lite model.
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    
    # Save the TF Lite model.
    with tf.io.gfile.GFile(name.replace('h5','tflite'), 'wb') as f:
      f.write(tflite_model)
        

In [4]:
cnn_network = CNNClassifier(n_epochs=10, verbose=True)
cnn_network.fit(X_train,y_train)

name = pre_name+"cnn.h5"    
cnn_network.model_.save(name)
convertH5TFlite(name)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 3)]          0         
                                                                 
 conv1d (Conv1D)             (None, 194, 6)            132       
                                                                 
 average_pooling1d (AverageP  (None, 64, 6)            0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 58, 12)            516       
                                                                 
 average_pooling1d_1 (Averag  (None, 19, 12)           0         
 ePooling1D)                                                     
                                                                 
 flatten (Flatten)           (None, 228)               0     

KeyboardInterrupt: 

In [ ]:
y_columns_walk_other_p = cnn_network.model_.predict(X_columns_walk_other)
print(confusion_matrix(y_columns_walk_other,np.argmax(y_columns_walk_other_p,axis=1) ))

y_valid_p = cnn_network.model_.predict(X_valid.transpose(0, 2, 1))
print(confusion_matrix(y_valid,np.argmax(y_valid_p,axis=1) ))



In [ ]:
'''
param_grid = {"kernel_size": [7, 9], "avg_pool_size": [3, 5]}
cnn_network = CNNClassifier(n_epochs=10, verbose=False)
grid = GridSearchCV(cnn_network, param_grid=param_grid, cv=3)
grid.fit(X_train, y_train)

print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_valid, y_valid)))
print("Best parameters: {}".format(grid.best_params_))
'''

In [ ]:
inception_network = InceptionTimeClassifier(n_epochs=10, verbose=True,n_filters = 32,batch_size = 8, depth = 2 )
inception_network.fit(X_train,y_train)
name = pre_name+"inception.h5"
inception_network.model_.save(name)
convertH5TFlite(name)


In [ ]:
y_columns_walk_other_p = inception_network.model_.predict(X_columns_walk_other)
print(confusion_matrix(y_columns_walk_other,np.argmax(y_columns_walk_other_p,axis=1) ))

y_valid_p = inception_network.model_.predict(X_valid.transpose(0, 2, 1))
print(confusion_matrix(y_valid,np.argmax(y_valid_p,axis=1) ))



In [ ]:
fcn_network = FCNClassifier(n_epochs=6, verbose=True,batch_size=32)
fcn_network.fit(X_train,y_train)

name = pre_name+"fcn.h5"
fcn_network.model_.save(name)
convertH5TFlite(name)

In [ ]:
y_columns_walk_other_p = fcn_network.model_.predict(X_columns_walk_other)
print(confusion_matrix(y_columns_walk_other,np.argmax(y_columns_walk_other_p,axis=1) ))

y_valid_p = fcn_network.model_.predict(X_valid.transpose(0, 2, 1))
print(confusion_matrix(y_valid,np.argmax(y_valid_p,axis=1) ))



In [5]:
resnet_network = ResNetClassifier(n_epochs=6, verbose=True,batch_size = 64)
resnet_network.fit(X_train,y_train)

name = pre_name+"resnet.h5"
resnet_network.model_.save(name)
convertH5TFlite(name)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200, 3)]     0           []                               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 200, 64)      1600        ['input_2[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 200, 64)     256         ['conv1d_2[0][0]']               
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 200, 64)      0           ['batch_normalization[0][0]

INFO:tensorflow:Assets written to: C:\Users\li_fr\AppData\Local\Temp\tmp866h9nio\assets


INFO:tensorflow:Assets written to: C:\Users\li_fr\AppData\Local\Temp\tmp866h9nio\assets


In [6]:
y_columns_walk_other_p = resnet_network.model_.predict(X_columns_walk_other)
print(confusion_matrix(y_columns_walk_other,np.argmax(y_columns_walk_other_p,axis=1) ))

y_valid_p = resnet_network.model_.predict(X_valid.transpose(0, 2, 1))
print(confusion_matrix(y_valid,np.argmax(y_valid_p,axis=1) ))



2336/2336 [==============================] - 37s 15ms/step
[[    0     0]
 [ 3617 71113]]
342/342 [==============================] - 5s 15ms/step
[[4041   29]
 [ 440 6415]]


In [7]:
macnn_network = MACNNClassifier(n_epochs=6, verbose=True,batch_size = 32,repeats=1,filter_sizes=[16,32,64],kernel_size=(3, 5, 7),reduction=4)
macnn_network.fit(X_train,y_train)

name = pre_name+"macnn.h5"
macnn_network.model_.save(name)
convertH5TFlite(name)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 200, 3)]     0           []                               
                                                                                                  
 conv1d_13 (Conv1D)             (None, 200, 16)      160         ['input_3[0][0]']                
                                                                                                  
 conv1d_14 (Conv1D)             (None, 200, 16)      256         ['input_3[0][0]']                
                                                                                                  
 conv1d_15 (Conv1D)             (None, 200, 16)      352         ['input_3[0][0]']                
                                                                                            

INFO:tensorflow:Assets written to: C:\Users\li_fr\AppData\Local\Temp\tmpsfcatmc7\assets


INFO:tensorflow:Assets written to: C:\Users\li_fr\AppData\Local\Temp\tmpsfcatmc7\assets


In [8]:
y_columns_walk_other_p = macnn_network.model_.predict(X_columns_walk_other)
print(confusion_matrix(y_columns_walk_other,np.argmax(y_columns_walk_other_p,axis=1) ))

y_valid_p = macnn_network.model_.predict(X_valid.transpose(0, 2, 1))
print(confusion_matrix(y_valid,np.argmax(y_valid_p,axis=1) ))

2336/2336 [==============================] - 22s 9ms/step
[[    0     0]
 [   62 74668]]
342/342 [==============================] - 3s 9ms/step
[[4028   42]
 [  76 6779]]


In [10]:
macnn_network2 = MACNNClassifier(n_epochs=6, verbose=True,batch_size = 32,repeats=1,filter_sizes=[8,16,32],kernel_size=(3, 5, 7),reduction=2)
macnn_network2.fit(X_train,y_train)

name = pre_name+"macnn2.h5"
macnn_network2.model_.save(name)
convertH5TFlite(name)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 200, 3)]     0           []                               
                                                                                                  
 conv1d_31 (Conv1D)             (None, 200, 8)       80          ['input_5[0][0]']                
                                                                                                  
 conv1d_32 (Conv1D)             (None, 200, 8)       128         ['input_5[0][0]']                
                                                                                                  
 conv1d_33 (Conv1D)             (None, 200, 8)       176         ['input_5[0][0]']                
                                                                                            

INFO:tensorflow:Assets written to: C:\Users\li_fr\AppData\Local\Temp\tmprskcfvg_\assets


INFO:tensorflow:Assets written to: C:\Users\li_fr\AppData\Local\Temp\tmprskcfvg_\assets


In [11]:
y_columns_walk_other_p = macnn_network2.model_.predict(X_columns_walk_other)
print(confusion_matrix(y_columns_walk_other,np.argmax(y_columns_walk_other_p,axis=1) ))

y_valid_p = macnn_network2.model_.predict(X_valid.transpose(0, 2, 1))
print(confusion_matrix(y_valid,np.argmax(y_valid_p,axis=1) ))

2336/2336 [==============================] - 19s 8ms/step
[[    0     0]
 [  280 74450]]
342/342 [==============================] - 3s 9ms/step
[[4007   63]
 [  73 6782]]
